# QuASAr API Usage Examples

This notebook demonstrates several ways to use the QuASAr library and verifies results on small circuits.

## Bell State with SimulationEngine

In [ ]:
import numpy as np
from quasar import Circuit, SimulationEngine, Backend

bell = Circuit([
    {"gate": "H", "qubits": [0]},
    {"gate": "CX", "qubits": [0, 1]},
])
engine = SimulationEngine()
result = engine.simulate(bell, backend=Backend.STATEVECTOR)
state = result.ssd.extract_state(0)
expected = (1/np.sqrt(2)) * np.array([1,0,0,1])
np.allclose(state, expected)

## Building from Qiskit

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

qc = QuantumCircuit(3)
qc.h(0)
qc.cx(0,1)
qc.cx(0,2)
quasar_circuit = Circuit.from_qiskit(qc)
result = engine.simulate(quasar_circuit, backend=Backend.STATEVECTOR)
state = result.ssd.extract_state(0)
expected = Statevector.from_instruction(qc).data
expected_le = expected.reshape([2]*3).transpose(*reversed(range(3))).reshape(-1)
np.allclose(state, expected_le)

## Direct Backend Usage

In [ ]:
from quasar import StatevectorBackend

backend = StatevectorBackend()
backend.load(2)
backend.apply_gate("H", [0])
backend.apply_gate("CX", [0,1])
backend.statevector()

## Circuit Analysis and Planning

In [ ]:
from quasar import CircuitAnalyzer, Planner

analyzer = CircuitAnalyzer(quasar_circuit)
estimates = analyzer.resource_estimates()
planner = Planner()
plan = planner.plan(quasar_circuit, backend=Backend.STATEVECTOR)
estimates, plan

## Random Circuit Comparison

In [ ]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

np.random.seed(0)
rc = QuantumCircuit(2)
rc.rx(0.3,0)
rc.ry(0.2,1)
rc.cz(0,1)
rc.rz(0.1,0)
rc.h(1)

rand_circuit = Circuit.from_qiskit(rc)
result = engine.simulate(rand_circuit, backend=Backend.STATEVECTOR)
quasar_state = result.ssd.extract_state(0)
expected = Statevector.from_instruction(rc).data
expected_le = expected.reshape([2]*2).transpose(*reversed(range(2))).reshape(-1)
np.allclose(quasar_state, expected_le)